In [28]:
from utils.processor import *
from utils.solver import *
from config import *
from utils.tom_api import *
import folium
from folium.features import DivIcon

'''
Timer class - (SOURCE CODE) https://realpython.com/python-timer/
*** We reference and use this code since we decided to simply use it for it's utility, and nothing more ***
'''
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = round(time.perf_counter() - self._start_time, 3)
        self._start_time = None
        print(f"Elapsed time: {elapsed_time} seconds")


In [29]:
timer = Timer()
timer.start()

data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
merged_datapath = os.path.join(data_dir, 'parking-meters_merged_twins.csv')

ttapi = TomSearchAPI(API_KEY)
topK = 10

'''
Specify which case you want to try, or feel free to make your own
'''
use_case = 1

if use_case == 1:
    #Marine Drive -> VGH
    origin = (MARINE_DRIVE['lat'], MARINE_DRIVE['lon'])
    dest = ttapi.find_location('Vancouver General Hospital')
    stay_duration = 2 * 60 
    parking_budget = 12.00
    
elif use_case == 2:
    #UBC (Ponderosa Commons) to Kitsilano Beach
    origin = (BROCK_COMMONS_TALLWOOD['lat'], BROCK_COMMONS_TALLWOOD['lon'])
    dest = ttapi.find_location('Kitsilano Beach')
    stay_duration = 1 * 60 
    parking_budget = 6.00
    
elif use_case == 3:
    #UBC (Brock Commons) to Downtown
    origin = (PONDEROSA_COMMONS_SPRUCE['lat'], PONDEROSA_COMMONS_SPRUCE['lon'])
    dest = ttapi.find_location('Pacific Center')
    stay_duration = 1.5 * 60 
    parking_budget = 8.00
    
elif use_case == 4:
    #UBC (University Village) to Metrotown
    origin = (UNI_VILLAGE_ANW['lat'], UNI_VILLAGE_ANW['lon'])
    dest = ttapi.find_location('Metrotown Chatime')
    stay_duration = 0.5 * 60 
    parking_budget = 5.00
    
else:
    raise ValueError(f'Invalid use-case: {use_case}')

print(dest)
dest = dest['location']
dest = (dest['lat'], dest['lon'])
dest

{'poi_name': 'Vancouver General Hospital', 'address': '855 West 12th Avenue, Vancouver BC V5Z 1M9', 'location': {'lat': 49.26132, 'lon': -123.12328}}


(49.26132, -123.12328)

In [30]:
'''
NOTE - If you are running it for the first time, please set load_merged=False
    - The script will merge all twin meters as 1 spot
    - It will also exclusively retain Single/Twin meterheads
    - It will save them to merged_datapath
IF you are running it again, and again, simply set load_merged=True to save the preprocessing time
'''
processor = Processor(datapath, merged_datapath, origin, dest, topK,
                      stay_duration, parking_budget, load_merged=True)
variables = processor.form_variables_dict()
variables

(3655, 24)
(3655, 25)
IDX - 0
Time limit: 2 	 Rate: 6.0
~~~~~~~~~~~~~~~~~~~~ Single ~~~~~~~~~~~~~~~~~~~~
*** Travel time: 16.233333333333334 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 1
Time limit: 2 	 Rate: 6.0
******************** Twin **********************
*** Travel time: 16.233333333333334 ***
out: 1 - in: 0
~~~ Parking Status: 3 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 12.0, 'savings': 0.0, 'expected_avail': 3, 'et_next_out': 10.0, 'et_next_in': 24.0, 'walk_time': 3.9, 'travel_et': 16.233333333333334}, 'meterid': "('832803', '832801')", 'latln': (49.26033994446587, -123.12438892783342)}
IDX - 2
Time limit: 2 	 Rate: 6.0
******************** Twin **********************
*** Travel time: 16.266666666666666 ***
out: 1 - in: 0
~~~ Parking Status: 3 ~~~
{'b_2': 1.0, 'constraint_coeffs': {'parking_fee': 12.0, 'savings': 0.0, 'expected_avail': 3, 'et_next_out': 14.0, 'et_next_in': 25.0, 'walk_time': 4.017, 'travel_et': 16.26666666

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 12.0,
   'savings': 0.0,
   'expected_avail': 3,
   'et_next_out': 10.0,
   'et_next_in': 24.0,
   'walk_time': 3.9,
   'travel_et': 16.233333333333334},
  'meterid': "('832803', '832801')",
  'latln': (49.26033994446587, -123.12438892783342)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 12.0,
   'savings': 0.0,
   'expected_avail': 3,
   'et_next_out': 14.0,
   'et_next_in': 25.0,
   'walk_time': 4.017,
   'travel_et': 16.266666666666666},
  'meterid': "('832804', '832806')",
  'latln': (49.260245481703116, -123.12422547036608)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 12.0,
   'savings': 0.0,
   'expected_avail': 3,
   'et_next_out': 8.0,
   'et_next_in': 23.0,
   'walk_time': 4.05,
   'travel_et': 16.266666666666666},
  'meterid': '832805',
  'latln': (49.26023253804825, -123.12439225873811)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 12.0,
   'savings': 0.0,
   'expecte

In [31]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

ParkingOptimizer:
MINIMIZE
1.0*y_1 + 1.0*y_2 + 1.0*y_3 + 1.0*y_4 + 1.0*y_5 + 0.0
SUBJECT TO
_C1: 3 y_1 + 3 y_2 + 3 y_3 + 3 y_4 + 3 y_5 >= 3

_C2:0 >= 0

_C3: - 16.2333333333 y_1 - 16.2666666667 y_2 - 16.2666666667 y_3
 - 16.3333333333 y_4 - 16.35 y_5 >= -16.29

_C4: - 3.9 y_1 - 4.017 y_2 - 4.05 y_3 - 4.233 y_4 - 4.317 y_5 >= -4.1034

_C5: - 10 y_1 - 14 y_2 - 8 y_3 - 11 y_4 - 14 y_5 >= -11.4

_C6: y_1 + y_2 + y_3 + y_4 + y_5 = 1

VARIABLES
y_1 <= 1 Continuous
y_2 <= 1 Continuous
y_3 <= 1 Continuous
y_4 <= 1 Continuous
y_5 <= 1 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/f5ca229980814bce8a93a29d037a0989-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/f5ca229980814bce8a93a29d037a0989-pulp.sol (default strategy 1)


,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
1,y_2,0.566667,12.0,3,16.266667,4.017,14.0,25.0
2,y_3,0.433333,12.0,3,16.266667,4.050,8.0,23.0
0,y_1,0.000000,12.0,3,16.233333,3.900,10.0,24.0
3,y_4,0.000000,12.0,3,16.333333,4.233,11.0,22.0
4,y_5,0.000000,12.0,3,16.350000,4.317,14.0,23.0


In [32]:
timer.stop()

Elapsed time: 13.084 seconds


# Visualization

In [33]:


fmap = folium.Map(location=dest, zoom_start=16, width="100%", height="50%")

# folium.Marker(
#     location=origin, icon=folium.Icon(color="red",icon="home", prefix='fa')
# ).add_to(fmap)

folium.Marker(
    location=dest, icon=folium.Icon(color="red",icon="location-dot", prefix='fa')
).add_to(fmap)


for i, spot in enumerate(soln.loc[soln.optimal_weight > 0]['var']):
    print(spot)
    rank = i+1
    spot_loc = variables[spot]['latln']
    folium.Marker(
        location = spot_loc,
        popup = f'Rank-{rank}',
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(7,20),
            html=f'<div style="font-size: 18pt; color : black">{rank}</div>',
            )
    ).add_to(fmap)
    fmap.add_child(folium.CircleMarker(spot_loc, radius=15))

fmap

y_2
y_3
